Ce code récupère les URL d'un sitemap. Il vérifie d'abord si le sitemap est gzippé et le décompresse si nécessaire.
Ensuite, il analyse le contenu XML du sitemap pour en extraire les URL. Le script fonctionne également avec les index de sitemaps.

In [4]:
import requests
import gzip
import xml.etree.ElementTree as ET
import pandas as pd
from io import BytesIO

def fetch_urls_from_sitemap(sitemap_url):
    urls = []
    response = requests.get(sitemap_url)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch sitemap: {sitemap_url}")

    content = response.content

    # Handle GZIP-compressed sitemaps
    if sitemap_url.endswith('.gz'):
        with gzip.GzipFile(fileobj=BytesIO(content)) as f:
            content = f.read()

    # Parse the XML content
    root = ET.fromstring(content)

    # Determine if it's a sitemap index or a sitemap
    namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
    if root.tag.endswith('sitemapindex'):
        # Sitemap index, fetch nested sitemaps
        for sitemap in root.findall('ns:sitemap', namespace):
            loc = sitemap.find('ns:loc', namespace).text
            urls.extend(fetch_urls_from_sitemap(loc))
    elif root.tag.endswith('urlset'):
        # Regular sitemap, extract URLs
        for url in root.findall('ns:url', namespace):
            loc = url.find('ns:loc', namespace).text
            urls.append(loc)

    return urls

### Call to function ###

sitemap_url = input("Entrer l'URL du sitemap : ")

try:
  extracted_urls = fetch_urls_from_sitemap(sitemap_url)
  print(f"Extraction de {len(extracted_urls)} URL.")

  # Save to a DataFrame
  df = pd.DataFrame(extracted_urls, columns=['URL'])

  # Export to Excel
  output_file = "urls_sitemap.xlsx"
  df.to_excel(output_file, index=False)
  print(f"URL sauvegardées : {output_file}.")

except Exception as e:
  print(f"Error: {e}")

Entrer l'URL du sitemap : https://api.disneylandparis.com/sitemaps/marketing/sitemap.xml
Extraction de 11417 URL.
URL sauvegardées : urls_sitemap.xlsx.
